In [1]:
import os
os.chdir('..')

from util import CONFIG
CONFIG.set_use_mpl_tables(True)

In [2]:
from math import log2
import numpy as np

from sim_circuit import QuantumRegister, QuantumCircuit


def phase_estimation_unitary(n, U, swap=True):
    assert(U.shape[0] == U.shape[1])
    m = int(log2(U.shape[0]))

    eigvals, eigvecs = np.linalg.eig(U)

    q = QuantumRegister(n)
    a = QuantumRegister(m)
    qc = QuantumCircuit(q, a) # ancilla is last

    qc.append_u(eigvecs, a)
    qc.report('eigenstate')

    for i in range(n):
        qc.h(q[i])

    for i in range(n):
        if swap:
            for _ in range(2**i):
                qc.c_append_u(U, q[i], a)
        else:
            # qubit reversal
            for _ in range(2**i):
                qc.c_append_u(U, q[n-1-i], a)
            # decreasing powers of 2
            # for _ in range(2**(n-1-i)):
            #     qc.c_append_u(U, q[i], a)

    qc.report('geometric_sequence_superposition')

    qc.append_u(np.conj(eigvecs.transpose()), a)

    qc.report('geometric_sequence')

    qc.iqft(q if swap else q[::-1], swap)
    qc.report('estimate')

    return qc

In [3]:
import scipy.stats
    
n = 3
m = 2
    
U = scipy.stats.unitary_group.rvs(2**m)
qc = phase_estimation_unitary(n, U, swap=True)

In [4]:
from math import pi

eigvals, _ = np.linalg.eig(U)
theta = np.angle(eigvals[0])
if theta < 0:
    theta += 2*pi

v = theta/pi*2**(n-1)
print('\nfrequency from eigenvalue', v)


frequency from eigenvalue 3.5469209067643117


In [5]:
from util import print_state_table

state = qc.run()
print_state_table(qc.run())


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        00000   0.1190 + i0.0402    0.1256       18.67°   ███                       0.0158
1        00001   0.1106 + i0.0967    0.1469       41.16°   ███                       0.0216
2        00010   0.0962 + i0.1941    0.2166       63.64°   █████                     0.0469
3        00011   0.0391 + i0.5788    0.5801       86.14°   █████████████             0.3366
4        00100   0.2233 - i0.6620    0.6986      -71.64°   ████████████████          0.4881
5        00101   0.1506 - i0.1724    0.2289      -48.14°   █████                     0.0524
6        00110   0.1349 - i0.0669    0.1506      -26.62°   ███                       0.0227
7        00111   0.1263 - i0.0085    0.1266       -3.15°   ███                       0.016 
8        01000   0.0000 + i0.0000    0.0                             

In [6]:
result = qc.measure(shots = 1000)

sorted_counts = sorted(result['counts'].items(), key = lambda item: item[1], reverse=True)
sorted_counts

[(4, 496), (3, 337), (5, 43), (2, 40), (1, 33), (6, 21), (7, 15), (0, 15)]

In [7]:
from math import sqrt 

top_two = sorted(sorted_counts[:2])

p_below, p_above = top_two[0][1], top_two[1][1]

decimal_estimate = sqrt(p_above)/(sqrt(p_below)+ sqrt(p_above))

estimate = top_two[0][0] + decimal_estimate
print('\nfrequency from eigenvalue', v, '\nfrequency from measurement', estimate, '\nerror', abs(v - estimate))


frequency from eigenvalue 3.5469209067643117 
frequency from measurement 3.548161837273175 
error 0.0012409305088634248


In [8]:
from util import all_close, cis, prod
from math import cos

def complex_sincd(n, v):
    N = 2 ** n
    return [prod(
        cos((v - k) * pi / 2 ** (j + 1)) * cis((v - k) * pi / 2 ** (j + 1))
        for j in range(n)) for k in range(2 ** n)]


assert all_close(state, complex_sincd(n, v))

In [9]:
def test_unitary_inverse():
    n = 3
    m = 2
    
    U = scipy.stats.unitary_group.rvs(2**m)
    qc = phase_estimation_unitary(n, U, swap=True)
    qci = qc.inverse()

    qc.append(qci, QuantumRegister(m+n))
    state = qc.run()
    tabulate_state(state)

    assert all_close(state, init_state(m+n))